In [ ]:
!pip install transformers
!pip3 install natten -f https://shi-labs.com/natten/wheels/cu118/torch2.0.0/index.html

In [ ]:
from transformers import OneFormerProcessor, OneFormerForUniversalSegmentation
from PIL import Image
import pandas as pd
from tqdm import tqdm
import numpy as np

import torch
import torch.nn as nn
import warnings
warnings.filterwarnings('ignore')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')



In [ ]:
from transformers import AutoImageProcessor, Mask2FormerForUniversalSegmentation

In [ ]:
processor = AutoImageProcessor.from_pretrained("facebook/mask2former-swin-large-cityscapes-semantic")
model = Mask2FormerForUniversalSegmentation.from_pretrained("facebook/mask2former-swin-large-cityscapes-semantic").to(device)

from google.colab import drive
drive.mount('/content/drive')



In [ ]:
cd /content/drive/MyDrive/open

In [ ]:

df = pd.read_csv('test.csv')
df.head()

def rle_encode(mask):
    pixels = mask.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

cityscapes_to_12 = {
    0: [0],
    1: [1],
    2: [2,3,4],
    3: [4,5],
    4: [5],
    5: [6],
    6: [7],
    7: [8],
    8: [10],
    9: [11],
    10: [12],
    11: [13,14,15,17]
}

submit = pd.read_csv('./sample_submission.csv')

result = []
for i in tqdm(range(len(df))):
    image = Image.open(df['img_path'][i])
    # image resize
    image = image.resize((960, 540))

    semantic_inputs = processor(images=image, return_tensors="pt")
    # semantic_inputs = processor(images=image, task_inputs=["semantic"], return_tensors="pt",do_resize = True, size=(960,540))

    for key in semantic_inputs.keys():
        semantic_inputs[key] = semantic_inputs[key].to(device)

    semantic_outputs = model(**semantic_inputs)

    # pass through image_processor for postprocessing
    predicted_semantic_map = processor.post_process_semantic_segmentation(semantic_outputs, target_sizes=[image.size[::-1]])[0]

    del semantic_inputs, semantic_outputs
    predicted_semantic_map_np = np.array(predicted_semantic_map.cpu().numpy())

    # convert to rle
    for key, value in cityscapes_to_12.items():
        key_mask = np.isin(predicted_semantic_map_np, value)
        if np.sum(key_mask) > 0:
            mask_rle = rle_encode(key_mask)
            result.append(mask_rle)
        else:
            result.append(-1)



In [ ]:
submit = pd.read_csv('./sample_submission.csv')
submit['mask_rle'] = result
submit

submit.to_csv('segformer_pretrain_submit.csv', index=False)

In [ ]:
import cv2

# Define a color mapping for each label
label_to_color = {
    0: (0, 255, 0),   # label 0 - Green
    1: (255, 0, 0),   # label 1 - Red
    2: (0, 0, 255),   # label 2 - Blue
    3: (255, 255, 0), # label 3 - Yellow
    4: (255, 165, 0), # label 4 - Orange
    5: (128, 0, 128), # label 5 - Purple
    6: (255, 192, 203), # label 6 - Pink
    7: (0, 255, 255), # label 7 - Cyan
    8: (165, 42, 42),  # label 8 - Brown
    9: (0, 128, 0),   # label 9 - Dark Green
    10: (0, 0, 0),    # label 10 - Black
    11: (255, 255, 255) # label 11 - White
}


# Load the test image
image_path = '/content/drive/MyDrive/open/test_image/TEST_0000.png'
image = cv2.imread(image_path)

# Define a function to decode RLE
def rle_decode(mask_rle, shape):
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    mask = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        mask[lo:hi] = 1
    return mask.reshape(shape)

# Apply masks to the image
for mask_rle in result:
    if mask_rle != -1:
        mask = rle_decode(mask_rle, image.shape[:2])
        unique_labels = np.unique(mask)
        for label in unique_labels:
            if label in label_to_color:
                mask_color = label_to_color[label]
                image[mask == label] = mask_color




In [ ]:
import cv2
import numpy as np

# Define a color mapping for each label
label_to_color = {
    0: (0, 255, 0),   # label 0 - Green
    1: (255, 0, 0),   # label 1 - Red
    2: (0, 0, 255),   # label 2 - Blue
    3: (255, 255, 0), # label 3 - Yellow
    4: (255, 165, 0), # label 4 - Orange
    5: (128, 0, 128), # label 5 - Purple
    6: (255, 192, 203), # label 6 - Pink
    7: (0, 255, 255), # label 7 - Cyan
    8: (165, 42, 42),  # label 8 - Brown
    9: (0, 128, 0),   # label 9 - Dark Green
    10: (0, 0, 0),    # label 10 - Black
    11: (255, 255, 255) # label 11 - White
}

# Define the total number of labels
num_labels = 12

# Load the test image
image_path = '/content/drive/MyDrive/open/test_image/TEST_0000.png'
image = cv2.imread(image_path)


# Define a function to decode RLE
def rle_decode(mask_rle, shape):
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    mask = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        mask[lo:hi] = 1
    return mask.reshape(shape)

# Create an empty array to store all masks
all_masks = np.zeros((image.shape[0], image.shape[1], num_labels), dtype=np.uint8)

# Apply masks to the image and store them in all_masks
for label in range(num_labels):
    mask_rle = result[label]  # Assuming result is a list of RLE masks for each label
    mask_rle = str(mask_rle)  # Ensure mask_rle is a string
    mask = rle_decode(mask_rle, image.shape[:2])
    all_masks[:,:,label] = mask



In [ ]:
from matplotlib import pyplot as plt

# Display individual masks
for label in range(num_labels):
    mask_color = label_to_color[label]
    label_mask = all_masks[:,:,label]
    labeled_image = np.zeros_like(image)
    labeled_image[label_mask == 1] = mask_color

    plt.imshow(labeled_image)
    plt.title(f'Mask for Label {label}')
    plt.show()

In [ ]:
# Create an empty array to store the combined mask
combined_mask = np.zeros((image.shape[0], image.shape[1], 3), dtype=np.float32)

# Combine masks
for label in range(num_labels):
    mask_color = np.array(label_to_color[label], dtype=np.float32)
    label_mask = all_masks[:, :, label][:, :, np.newaxis]  # Add a new axis for broadcasting
    combined_mask += label_mask * mask_color

# Normalize combined mask
combined_mask = (combined_mask / np.max(combined_mask) * 255).astype(np.uint8)

# Display the combined mask
plt.imshow(combined_mask)
plt.title('Combined Mask')
plt.show()

# Define the desired output size (e.g., 960x540)
output_size = (960, 540)

# Resize the combined mask to the desired output size
combined_mask_resized = cv2.resize(combined_mask, output_size)

# Display the resized combined mask
plt.imshow(combined_mask_resized)
plt.title('Resized Combined Mask')
plt.show()
